In [1]:
from selenium import webdriver
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time
import pandas as pd

def scrape_acha_dynamic_stats():
    firefox_options = Options()
    firefox_options.add_argument("--headless")  

    geckodriver_path = r"C:\Program Files\GeckoDriver\geckodriver.exe"
    service = Service(executable_path=geckodriver_path)

    with webdriver.Firefox(service=service, options=firefox_options) as driver:
        url = "https://www.achahockey.org/stats/player-stats/all-teams/46?conference=10&division=-1&playertype=skater&position=skaters&rookie=no&sort=points&statstype=standard&page=1&league=1"
        driver.get(url)

        all_data = []
        
        while True:
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, "table")))
            
            page_source = driver.page_source
            soup = BeautifulSoup(page_source, 'html.parser')
            
            table = soup.find('table')
            if table:
                rows = table.find_all('tr')
                for i, row in enumerate(rows[1:], start=1):  
                    cols = row.find_all('td')
                    cols = [ele.text.strip() for i, ele in enumerate(cols) if i != 0 and i != 2 and i != len(cols) - 1]
                    all_data.append(cols)

            try:
                next_button = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.XPATH, "//a[contains(text(), 'Next')]"))
                )
                next_button.click()
                time.sleep(2)
            except:
                break

    return all_data

acha_dynamic_stats = scrape_acha_dynamic_stats()

column_names = ['Jersey#', 'Name', 'Pos', 'Division', 'Team', 'GP', 'G', 'A', 'PTS', 'Pt/G', 'PPG', 'SHG', 'GWG', 'SOGW', 'PIM', 'SH%']

df = pd.DataFrame(acha_dynamic_stats, columns=column_names)

# Skip every other row, keeping only odd-indexed rows (1, 3, 5, etc.)
df = df.iloc[::2]
df = df[df['GP'].notna()]

numeric_columns = ['Jersey#', 'GP', 'G', 'A', 'PTS', 'Pt/G', 'PPG', 'SHG', 'GWG', 'SOGW', 'PIM', 'SH%']
for column in numeric_columns:
    df[column] = pd.to_numeric(df[column], errors='coerce')  

string_columns = ['Name', 'Pos', 'Division', 'Team']
for column in string_columns:
    df[column] = df[column].astype(str)
    


print(df)

print("\nDataFrame Info:")
print(df.info())

df.to_csv('acha_md3_skaters.csv', index=False)

      Jersey#                Name Pos                  Division  \
0        11.0       Brendan Danou   F         MD3 MCHC(MI) East   
2         5.0        Brett Oberle   F           MD3 Independent   
4        17.0        Jake Onstott   F         MD3 MCHC(MI) West   
6        27.0    Andrew Fenninger   C             MD3 MCHC (NE)   
8        24.0       Aaron Higgins   F  MD3 NECHA Colonial North   
...       ...                 ...  ..                       ...   
5780     27.0         Daniel Seim   F     MD3 MACHA Silver West   
5782     25.0     Jason Thornberg   F         MD3 MCHC(MI) West   
5784     17.0  Garrett Vanderlaan   D         MD3 MCHC(MI) West   
5786     44.0        Matthew Page   G                   MD3 CHE   
5788     28.0    Johnathan Konett   D         MD3 MCHC(MI) East   

                                        Team  GP   G   A  PTS  Pt/G  PPG  SHG  \
0                     MD3 Oakland University  17  21  23   44  2.59    3  0.0   
2        MD3 United States Air Fo